<a href="https://colab.research.google.com/github/Frenz86/machine-learning-class/blob/main/Esercizio_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regression con piu input

* Numero di input > 1 --> 13 input
* Numero di output 1 

> $$y = w_0x_0 + w_1x_1 + w_2x_2 + w_3x_3 + w_4x_4 + w_5x_5 + w_6x_6 +w_7x_7 + w_8x_8 + w_9x_9 + w_{10}x_{10} +w_{11}x_{11} + w_{12}x_{12} + w_{13}x_{13} \quad x_0=1 $$

* parametri ($w$)  = numero di input + 1 ( $w_0$ intercept --> condizioni iniziali)

* Collezonamento dati

| $x_0$   |  $x_1$ numero di letti    | $x_2$ $m^2$ |  altro  |     y | $\hat{y}$ | 
| --  | ------------------------- | ----------- |----------- | ---| --|
 1  | 3                         | 100         | altro      |  100| 99.3|
 1  | 1                         | 100         | altro      |  50|  49.4|
 1  | 5                         | 70         | altro       |  110| 112.3|
 1  | 6                         | 60         | altro       |  80| 87|
 1  | 2                         | 40         | altro       |  30|  31 |

* $Y = XW \quad W = (X^TX)^{-1}X^TY  $ 
* $\hat{Y} = XW$
* $e =\frac{(Y-\hat{Y})^2}{2n}$ Mean square error 


> Su nuovi dati come performera il nostro algoritmo

In [93]:
 from sklearn.datasets import load_boston, load_diabetes
 from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, TheilSenRegressor, RANSACRegressor
 from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
 from sklearn.metrics import mean_squared_error, mean_absolute_error
 from sklearn.model_selection import train_test_split
 import pandas as pd
 import numpy as np
 import plotly.graph_objects as go
 from sklearn.pipeline import Pipeline
 from sklearn.preprocessing import PolynomialFeatures,StandardScaler
 from joblib import dump, load

In [4]:
boston = load_boston()
#diabetes = load_diabetes()
#X = boston['data']
Y = boston['target']
#Y.shape
column_names = []
i = 1
for v in list(boston["feature_names"]):
    l = "x" + str(i) + "-" + v
    column_names.append(l)
    i = i+1
column_names

['x1-CRIM',
 'x2-ZN',
 'x3-INDUS',
 'x4-CHAS',
 'x5-NOX',
 'x6-RM',
 'x7-AGE',
 'x8-DIS',
 'x9-RAD',
 'x10-TAX',
 'x11-PTRATIO',
 'x12-B',
 'x13-LSTAT']

In [5]:
df_boston = pd.DataFrame(boston.data,columns=column_names)
df_boston["Y"] = Y
df_boston

,x1-CRIM,x2-ZN,x3-INDUS,x4-CHAS,x5-NOX,x6-RM,x7-AGE,x8-DIS,x9-RAD,x10-TAX,x11-PTRATIO,x12-B,x13-LSTAT,Y
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


In [27]:
X_train, X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.35, random_state=42, shuffle=True) # Migliri
poly = PolynomialFeatures(degree=4)
poly.fit(X_train)
X_train_new = poly.transform(X_train)
reg = LinearRegression()
reg.fit(X_train_new,Y_train)

X_test_new =poly.transform(X_test)
reg.predict(X_test_new,Y_test)

(328, 2380) (328, 13)


In [96]:
# Preprocessing
X = df_boston.iloc[:,:-1].values
Y = df_boston["Y"].values # array
X_train, X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.35, random_state=26, shuffle=True) # Migliri

In [97]:
# Training X_train -------------------------
#reg = RandomForestRegressor()  # instanzia la classe
#reg = GradientBoostingRegressor()
#reg = LinearRegression()
#reg.fit(X_train,Y_train) # allenamento
reg = Pipeline([
    ("std", StandardScaler())  ,          
    ("poly",PolynomialFeatures(degree=3,interaction_only=False)),
	#("reg", Lasso())
 	("reg", ElasticNet())
    #("reg", TheilSenRegressor())
    #("reg", LinearRegression())
])
reg.fit(X_train,Y_train)
#print(np.round(reg.named_steps["reg"].coef_,3),reg.named_steps["reg"].intercept_)
#---------------------------------
dump(reg, 'mypipeline.joblib')

regload = load("mypipeline.joblib")
# Training Error
Y_hat_train = regload.predict(X_train)
err_sklearn_rmse = np.sqrt(mean_squared_error(Y_train,Y_hat_train))
err_sklearn_mae = mean_absolute_error(Y_train,Y_hat_train)
err_mae = np.sum(np.abs(Y_hat_train -Y_train))/len(Y_hat_train) # mean absolute error ()
err_rmse = np.sqrt(np.sum(np.power(Y_hat_train -Y_train,2))/len(Y_hat_train)) # mean squared error ()
print("Training sklaern mse ", err_sklearn_rmse)
print("Training  err mae", np.round(err_mae,3))
print("Training err rmse ", np.round(err_rmse,3)) 

# Testing Error
Y_hat_test = regload.predict(X_test)
err_sklearn_rmse = np.sqrt(mean_squared_error(Y_test,Y_hat_test))
err_sklearn_mae = mean_absolute_error(Y_test,Y_hat_test)
err_mae = np.sum(np.abs(Y_hat_test -Y_test))/len(Y_hat_test) # mean absolute error ()
err_rmse = np.sqrt(np.sum(np.power(Y_hat_test -Y_test,2))/len(Y_hat_test)) # mean squared error ()

print("Test sklaern mse ", err_sklearn_rmse)
print("Test  err mae", np.round(err_mae,3))
print("Test err rmse ", np.round(err_rmse,3)) 

Training sklaern mse  3.400338407411902
Training  err mae 2.491
Training err rmse  3.4
Test sklaern mse  4.027200818467504
Test  err mae 2.662
Test err rmse  4.027


In [84]:
# Figure Test Results
xaxis = np.linspace(1,len(Y_train),len(Y_train),dtype=np.int)
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=xaxis, y=Y_train,mode='lines+markers', name="Y Train")
)
fig.add_trace(
    go.Scatter(x=xaxis, y=Y_hat_train,mode='lines+markers', name="Y Hat Train")
)
fig.update_layout(hovermode="x", title="Train Results", xaxis_title="Samples",yaxis_title="Price")
fig.show()

In [85]:
# Figure Test Results
xaxis = np.linspace(1,len(Y_test),len(Y_test),dtype=np.int)
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=xaxis, y=Y_test,mode='lines+markers', name="Y Test")
)
fig.add_trace(
    go.Scatter(x=xaxis, y=Y_hat_test,mode='lines+markers', name="Y Hat Test")
)
fig.update_layout(hovermode="x", title="Test Results", xaxis_title="Samples",yaxis_title="Price")
fig.show()

In [ ]:
# Plot Esercizio

# 1) Esercizio Train Test error function
# 2) Grafico plotly --> Y_pred, Y test set



In [ ]:
df_boston["Y_hat"] = Y_hat
df_boston.to_csv("test.csv", index=None)

In [ ]:
df_boston

,x1-CRIM,x2-ZN,x3-INDUS,x4-CHAS,x5-NOX,x6-RM,x7-AGE,x8-DIS,x9-RAD,x10-TAX,x11-PTRATIO,x12-B,x13-LSTAT,Y,Y_hat
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,25.133
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,22.159
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,34.404
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,33.984
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,35.341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4,23.268
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6,20.094
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9,25.214
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0,23.514


In [ ]:
# Training set (X)

In [ ]:
# Su n

In [ ]:
from sklearn.pipeline import Pipeline
import pandas as pd
from sklearn import datasets

In [ ]:
def load_dataset(name="boston"):
    datasets_list = ["boston", "diabetes"] #,"tips"]
    chosen_dataset = name #"boston"

    boston = datasets.load_boston()
    df_boston = pd.DataFrame(boston.data,columns=boston.feature_names)
    df_boston["price"] = boston.target

    diabetes = datasets.load_diabetes()
    df_diabetes = pd.DataFrame(diabetes.data,columns=diabetes.feature_names)
    df_diabetes["desease"] = diabetes.target
    df_diabetes = df_diabetes.round(decimals=5)

    #df_nuovo_dataset = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vSqHhx2kS9gCNmI04yksqTP2PRsT6ifTU2DLokKs3Y6KgcSGIAL7_4t_q_8kNhVkFA0xD2nt7hn_w-5/pub?output=csv")

    dict_datasets = {
        "boston": df_boston,
        "diabetes": df_diabetes,
    #    "nuovo" : df_nuovo_dataset
    }    
    df = dict_datasets[chosen_dataset]#.head(10)
    df = df.dropna()

    #print(df.head())
    X = df.iloc[:,0:-1].values  # Input: dalla prima alla penultima colonna
    Y = df.iloc[:,-1].values    # Target: utlima colonna

    return X,Y,df

In [ ]:
X,Y,df = load_dataset(name="boston")
display(df.head())

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,price
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Divido i dati in train e test
2. Preceprocessing: Lavoro con gli input. Standard Scaler, Polinomial?
3. Alleno il modello. Con o senza pipeline
4. Testo il modello (Su dati che non ho usato nel training)



## Soluzione

In [ ]:
from sklearn.linear_model import LinearRegression,SGDRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, shuffle=True, random_state=42)
#print(X_train.shape, X_test.shape)

(354, 13) (152, 13)


In [ ]:
reg = LinearRegression(fit_intercept=True)
reg.fit(X_train,y_train)
y_hat_train = reg.predict(X_train)
mse = mean_squared_error(y_train, y_hat_train)
R2 = reg.score(X_train, y_train)
print(mse,R2)
xaxis = np.linspace(1,len(y_train),len(y_train),dtype=np.int)
#plt.plot(xaxis, y_train)
#plt.plot(xaxis,y_hat_train)

fig = go.Figure()
fig.add_trace(
    go.Scatter(x=xaxis, y=y_train,mode='lines+markers', name="Y train")
)
fig.add_trace(
    go.Scatter(x=xaxis, y=y_hat_train,mode='lines+markers', name="Y Hat train")
)
fig.show()

22.545481487421423 0.7434997532004697


In [ ]:
y_hat_test = reg.predict(X_test)
mse = mean_squared_error(y_test, y_hat_test)
R2 = reg.score(X_test, y_test)
print(mse,R2)
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=xaxis, y=y_test,mode='lines+markers', name="Y train")
)
fig.add_trace(
    go.Scatter(x=xaxis, y=y_hat_test,mode='lines+markers', name="Y Hat train")
)
fig.show()

21.517444231176903 0.7112260057484974


In [ ]:
from sklearn.pipeline import Pipeline
from joblib import dump, load
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression,SGDRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split

generate_random = np.random.RandomState(667)
x = 10 * generate_random.rand(100)
y = 3 *np.sin(x) + np.random.randn(100) # -1 0 1

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, shuffle=True, random_state=42)

# creazione del modello
X_train = X_train.reshape(-1,1) # 100x1
mypipe = Pipeline([
	("std",  StandardScaler() ),  # (x-np.mean(x))/np.std(x) -----> z = (x - x_mean) / std
    ("poly",  PolynomialFeatures(degree=4) ),
	("reg", LinearRegression(fit_intercept=True))
])
mypipe.fit(X_train,y_train)
print(mypipe.named_steps["reg"].coef_,mypipe.named_steps["reg"].intercept_)
Y_hat = mypipe.predict(X_train)
dump(mypipe, 'mypipeline.joblib')

fig = plt.figure(figsize=(15,10))
plt.scatter(X_train, y_train)
plt.scatter(X_train, Y_hat)
plt.show()


In [ ]:
from joblib import dump, load
# ------------
# Produzione
loaded_pipe = load('mypipeline.joblib')
X_test = X_test.reshape(-1,1)  
Y_hat_test = loaded_pipe.predict(X_test)

fig = plt.figure(figsize=(15,10))
plt.scatter(X_test,y_test)
plt.scatter(X_test, Y_hat_test)
plt.show()